In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import gc
import re
import time
import warnings
warnings.filterwarnings("ignore")

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']

invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt'), header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']

answer_info = pd.read_table(os.path.join(DATA_PATH, 'answer_info_0926.txt'), header=None)
answer_info.columns = ['回答ID','问题ID','用户ID', '回答创建时间' ,'回答内容的单字编码序列', '回答内容的切词编码序列' ,'回答是否被标优', '回答是否被推荐' ,'回答是否被收入圆桌', '是否包含图片' ,'是否包含视频', '回答字数' ,'点赞数', '取赞数' ,'评论数' ,'收藏数', '感谢数' ,'举报数', '没有帮助数' ,'反对数']

oversample = False

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)

topic_vectors = pd.read_table(os.path.join(DATA_PATH, 'topic_vectors_64d.txt'), header=None)
topic_vectors.columns = ['话题ID编码序号','Topic']
topic_vectors['Topic'] = topic_vectors['Topic'].apply(lambda x: [float(num) for num in x.split(' ')])

embedding = np.vstack(list(topic_vectors['Topic']))
embedding = np.vstack([np.zeros((1, 64)), embedding])

print("Used time: %d s" % (time.time()-tic))

Used time: 97 s


In [3]:
tic = time.time()
tqdm.pandas()
question_info['问题绑定的话题ID'] = question_info['问题绑定的话题ID'].parallel_apply(lambda x: ','.join([ num[1:] for num in x.split(',')]))
answer_info = answer_info.merge(question_info[['问题ID','问题绑定的话题ID']], on='问题ID', how='left')

print("Used time: %d s" % (time.time()-tic))

Used time: 27 s


In [4]:
tic = time.time()
tmp = data['邀请创建时间'].apply(lambda x : x.split('-'))
data['邀请创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
data['邀请创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))

tmp = answer_info['回答创建时间'].apply(lambda x : x.split('-'))
answer_info['回答创建时间_H'] = tmp.apply(lambda x : int(x[1::1][0][1:]))
answer_info['回答创建时间_D'] = tmp.apply(lambda x : int(x[::2][0][1:]))


data['id'] = np.arange(len(data))
answer_info['atime'] = answer_info['回答创建时间_D'] * 24 + answer_info['回答创建时间_H']
data['itime'] = data['邀请创建时间_D'] * 24 + data['邀请创建时间_H']
inv = data[['id', '用户ID', 'itime']]
ans = answer_info[['问题绑定的话题ID','回答ID','问题ID','用户ID','回答创建时间_D','回答创建时间_H','atime','回答内容的单字编码序列','回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐' ,'回答是否被收入圆桌', '是否包含图片' ,'是否包含视频', '回答字数' ,'点赞数', '取赞数' ,'评论数' ,'收藏数', '感谢数' ,'举报数', '没有帮助数' ,'反对数']]
inv.sort_values(by=['用户ID', 'itime'], inplace=True)
ans.sort_values(by=['用户ID', 'atime'], inplace=True)

inv.columns = ['id', '用户ID','time']
ans.columns = ['问题绑定的话题ID','回答ID','问题ID','用户ID','回答创建时间_D','回答创建时间_H', 'time','回答内容的单字编码序列', '回答内容的切词编码序列','回答是否被标优', '回答是否被推荐' ,'回答是否被收入圆桌', '是否包含图片' ,'是否包含视频', '回答字数' ,'点赞数', '取赞数' ,'评论数' ,'收藏数', '感谢数' ,'举报数', '没有帮助数' ,'反对数']

tmp = pd.concat([inv, ans])
tmp.sort_values(by=['用户ID', 'time'], inplace=True)

print("Used time: %d s" % (time.time()-tic))

Used time: 41 s


In [5]:
tic = time.time()
def last_ans_stats(tmp):
    last_author = '-1'
    ts = []
    for id ,author, ti, answer_words in tqdm(tmp.values):
        if author != last_author:
            answer_list = []
            if pd.isna(id):
                answer_list.append(answer_words)
            ts.append([0])

        elif pd.isna(id):
            if len(answer_list)!=0:
                ts.append(list(answer_list))
            else:
                ts.append([0])
            answer_list.append(answer_words)

        else:
            if len(answer_list)!=0:
                ts.append(list(answer_list))
            else:
                ts.append([0])

        last_author = author

    tmp['answer_words'] = ts
    return tmp['answer_words']

tmpp = tmp[['id','用户ID','time','问题绑定的话题ID']]
res = last_ans_stats(tmpp)
tmp['曾回答问题绑定的话题ID'] = res

tmp = tmp[~tmp['id'].isnull()]
tmp.sort_values(by='id', inplace=True)

print("Used time: %d s" % (time.time()-tic))


Used time: 39 s


In [6]:
tic = time.time()
tqdm.pandas()
tmp['曾回答问题绑定的话题ID'] = tmp['曾回答问题绑定的话题ID'].progress_apply(lambda x: [0] if x[0]==0 else (','.join(x)))
tmp['曾回答问题绑定的话题ID'] = tmp['曾回答问题绑定的话题ID'].progress_apply(lambda x: [0] if x[0]==0 else (list(set([ int(num) for num in x.split(',')]))))
tmp['曾回答问题绑定的话题ID_w2v_mean'] = tmp['曾回答问题绑定的话题ID'].progress_apply(lambda x: np.mean(embedding[x], axis=0))

print("Used time: %d s" % (time.time()-tic))


Used time: 60 s


In [7]:
tic = time.time()

question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt'), header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']

topic_vectors = pd.read_table(os.path.join(DATA_PATH, 'topic_vectors_64d.txt'), header=None)
topic_vectors.columns = ['话题ID编码序号','Topic']
temp = topic_vectors.Topic.apply(lambda x: x.split(' '))
temp = pd.DataFrame(list(temp), columns=['T_{}'.format(i) for i in range(1,65)]) 
del topic_vectors['Topic']
topic_vectors = pd.concat([topic_vectors, temp], axis=1)

vocabulary = dict((topic_vectors['话题ID编码序号'][i], i) for i in range(len(topic_vectors)))

from sklearn.feature_extraction.text import CountVectorizer

cnt_vct = CountVectorizer(token_pattern='\\w+\\d+', binary=True, lowercase=False, vocabulary=vocabulary)
cnt_vct.fit(question_info['问题绑定的话题ID'])

A = cnt_vct.transform(question_info['问题绑定的话题ID'])

from scipy import sparse
B = topic_vectors.values[:, 1:].astype(float)
B = sparse.csr_matrix(B)

C = A.dot(B)
print(C.shape)

temp = pd.DataFrame(C.toarray())
temp.columns = ['问题绑定的话题ID_w2v_sum_{}'.format(i) for i in range(1,65)]
temp['问题ID'] = question_info['问题ID']

data = data.merge(temp, on='问题ID', how='left').merge(question_info, how='left', on='问题ID')

print("Used time: %d s" % (time.time()-tic))

(1829900, 64)
Used time: 56 s


In [8]:
tic = time.time()
data['问题绑定的话题ID_len'] = data['问题绑定的话题ID'].apply(lambda x: len(x.split(',')))

for i in range(1,65):
    data['问题绑定的话题ID_w2v_sum_{}'.format(i)] = data['问题绑定的话题ID_w2v_sum_{}'.format(i)] / data['问题绑定的话题ID_len']
    
ques_topic_feat = ['问题绑定的话题ID_w2v_sum_{}'.format(i) for i in range(1,65)]
ques_topic_feat = data[ques_topic_feat].values

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

sim = []
for i in tqdm(range(ques_topic_feat.shape[0])):
    sim.append(cosine(ques_topic_feat[i],tmp['曾回答问题绑定的话题ID_w2v_mean'][i]))
    
print("Used time: %d s" % (time.time()-tic))


Used time: 70 s


In [9]:
np.save(os.path.join(FEAT_PATH, 'prev_ans_ques_topic_sim.npy'), sim)